In [33]:
import torch
from abc import ABC, abstractmethod

class BaseBayesianOptimization(ABC):
    def __init__(self, bounds):
        self.bounds = bounds
        self.train_X = None
        self.train_Y = None
        self.model = None

    @abstractmethod
    def initialize(self, X_init, Y_init):
        pass

    @abstractmethod
    def _fit_model(self):
        pass

    @abstractmethod
    def acquisition_function(self):
        pass

    @abstractmethod
    def optimize_acquisition(self, acq_function):
        pass

    @abstractmethod
    def step(self):
        pass

    @abstractmethod
    def update(self, new_X, new_Y):
        pass

class BayesianOptimization(BaseBayesianOptimization):
    def __init__(self, bounds, beta=2.0):
        super().__init__(bounds)
        self.beta = beta

    def initialize(self, X_init, Y_init):
        self.train_X = X_init
        self.train_Y = Y_init
        self.model = self._fit_model()

    def _fit_model(self):
        from botorch.models import SingleTaskGP
        from gpytorch.mlls import ExactMarginalLogLikelihood
        from botorch.fit import fit_gpytorch_mll
        
        model = SingleTaskGP(self.train_X, self.train_Y)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)
        return model
    
    def acquisition_function(self):
        from botorch.acquisition import UpperConfidenceBound
        
        UCB = UpperConfidenceBound(self.model, beta=self.beta)
        return UCB

    def optimize_acquisition(self, acq_function):
        from botorch.optim import optimize_acqf
        
        candidates, _ = optimize_acqf(
            acq_function,
            bounds=self.bounds,
            q=1,
            num_restarts=10,
            raw_samples=20,
        )
        return candidates

    def step(self):
        acq_function = self.acquisition_function()
        new_X = self.optimize_acquisition(acq_function)
        return new_X

    def update(self, new_X, new_Y):
        self.train_X = torch.cat([self.train_X, new_X], dim=0)
        self.train_Y = torch.cat([self.train_Y, new_Y], dim=0)
        self.model = self._fit_model()

# Example Usage
if __name__ == "__main__":
    bounds = torch.tensor([[0.0], [1.0]])
    X_init = torch.rand(5, 1)
    Y_init = torch.sin(X_init * (2 * torch.pi))

    bo = BayesianOptimization(bounds=bounds)
    bo.initialize(X_init, Y_init)

    for i in range(10):
        new_X = bo.step()
        new_Y = torch.sin(new_X * (2 * torch.pi))
        bo.update(new_X, new_Y)
        print(f"Iteration {i+1}: Suggested point: {new_X.numpy()}, Function value: {new_Y.numpy()}")

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7541]), mean = tensor([-0.1095])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help 

Iteration 1: Suggested point: [[0.]], Function value: [[0.]]
Iteration 2: Suggested point: [[0.23685773]], Function value: [[0.9965926]]
Iteration 3: Suggested point: [[0.2712205]], Function value: [[0.9911244]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7415]), mean = tensor([0.0641])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

Iteration 4: Suggested point: [[0.2510879]], Function value: [[0.99997663]]
Iteration 5: Suggested point: [[0.2524308]], Function value: [[0.99988335]]
Iteration 6: Suggested point: [[0.24982424]], Function value: [[0.9999994]]
Iteration 7: Suggested point: [[0.2521402]], Function value: [[0.9999096]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7547]), mean = tensor([0.3440])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

Iteration 8: Suggested point: [[0.2412628]], Function value: [[0.9984935]]
Iteration 9: Suggested point: [[0.25649184]], Function value: [[0.9991682]]
Iteration 10: Suggested point: [[0.2513559]], Function value: [[0.9999637]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.6991]), mean = tensor([0.5312])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

In [38]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood

class BaseBayesianOptimization(ABC):
    def __init__(self, bounds):
        self.bounds = bounds
        self.train_X = None
        self.train_Y = None
        self.model = None

    @abstractmethod
    def initialize(self, X_init, Y_init):
        pass

    @abstractmethod
    def _fit_model(self):
        pass

    @abstractmethod
    def acquisition_function(self):
        pass

    @abstractmethod
    def optimize_acquisition(self, acq_function):
        pass

    @abstractmethod
    def step(self):
        pass

    @abstractmethod
    def update(self, new_X, new_Y):
        pass

class DiscreteBO(BaseBayesianOptimization):
    def __init__(self, bounds, beta=2.0, beta_h=10.0, l_h=2.0):
        super().__init__(bounds)
        self.beta = beta
        self.beta_h = beta_h
        self.l_h = l_h
        self.l = 1.0

    def initialize(self, X_init, Y_init):
        self.train_X = X_init
        self.train_Y = Y_init
        self.model = self._fit_model()

    def _fit_model(self):
        model = SingleTaskGP(self.train_X, self.train_Y)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)
        return model

    def acquisition_function(self, beta):
        return UpperConfidenceBound(self.model, beta=beta)

    def optimize_acquisition(self, acq_function):
        candidates, _ = optimize_acqf(
            acq_function,
            bounds=self.bounds,
            q=1,
            num_restarts=10,
            raw_samples=20,
        )
        return candidates

    def step(self):
        acq_function = self.acquisition_function(self.beta)
        new_X = self.optimize_acquisition(acq_function)
        return new_X

    def update(self, new_X, new_Y):
        self.train_X = torch.cat([self.train_X, new_X], dim=0)
        self.train_Y = torch.cat([self.train_Y, new_Y], dim=0)
        self.model = self._fit_model()

    def adjust_beta_and_l(self):
        # Define the optimization problem to adjust beta and l
        def objective(params):
            delta_beta, l = params
            adjusted_beta = self.beta + delta_beta
            acq_function = self.acquisition_function(adjusted_beta)
            new_X = self.optimize_acquisition(acq_function)
            rounded_new_X = torch.round(new_X)

            penalty = float('inf')
            if (rounded_new_X == self.train_X).all(dim=1).any():
                penalty = 1000  # Arbitrary high value to avoid repetition

            return delta_beta + torch.norm(new_X - rounded_new_X).item() + penalty

        # Initial guess and bounds for delta_beta and l
        initial_guess = torch.tensor([0.0, self.l])
        bounds = [(0.0, self.beta_h), (1e-3, self.l_h)]

        # Optimize the objective function
        result = torch.minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        delta_beta, l = result.x
        self.beta += delta_beta
        self.l = l

        print()
        print()
        print()
        print()
        print()
        print()
        print()

# Example Usage
if __name__ == "__main__":
    bounds = torch.tensor([[0.0], [1.0]])
    X_init = torch.rand(5, 1)
    Y_init = torch.sin(X_init * (2 * torch.pi))

    bo = DiscreteBO(bounds=bounds)
    bo.initialize(X_init, Y_init)

    for i in range(10):
        new_X = bo.step()
        if (new_X == bo.train_X).all(dim=1).any():
            bo.adjust_beta_and_l()
            new_X = bo.step()
        new_Y = torch.sin(new_X * (2 * torch.pi))
        bo.update(new_X, new_Y)
        print(f"Iteration {i+1}: Suggested point: {new_X.numpy()}, Function value: {new_Y.numpy()}")

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.8351]), mean = tensor([0.4630])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help

Iteration 1: Suggested point: [[0.]], Function value: [[0.]]
Iteration 2: Suggested point: [[0.22848748]], Function value: [[0.9908788]]
Iteration 3: Suggested point: [[0.2544213]], Function value: [[0.9996142]]
Iteration 4: Suggested point: [[0.25276524]], Function value: [[0.9998491]]


/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/optim/optimize.py:564: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = te

Iteration 5: Suggested point: [[0.2524194]], Function value: [[0.9998845]]
Iteration 6: Suggested point: [[0.25098068]], Function value: [[0.99998105]]
Iteration 7: Suggested point: [[0.25091198]], Function value: [[0.9999836]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.6101]), mean = tensor([0.6921])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help

Iteration 8: Suggested point: [[0.25210455]], Function value: [[0.99991256]]
Iteration 9: Suggested point: [[0.2525134]], Function value: [[0.9998753]]
Iteration 10: Suggested point: [[0.251264]], Function value: [[0.99996847]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.5722]), mean = tensor([0.7361])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help

In [3]:
import optuna
from optuna.integration import BoTorchSampler
from botorch.test_functions import Branin
import torch

# テスト関数の定義 (Branin 関数)
def branin(x):
    return Branin()(torch.tensor(x, dtype=torch.float)).item()

# 目的関数
def objective(trial):
    # 最適化するパラメータの範囲を設定
    x1 = trial.suggest_float('x1', -5, 10)
    x2 = trial.suggest_float('x2', 0, 15)
    # テスト関数の評価
    return branin([x1, x2])

# BoTorchSampler を使った最適化
sampler = BoTorchSampler()

# Optuna の study を作成して最適化を実行
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials=50)

# 結果の表示
print("Best value: ", study.best_value)
print("Best params: ", study.best_params)

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/286698348.py:19: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = BoTorchSampler()
[I 2024-08-01 13:04:44,459] A new study created in memory with name: no-name-3ef43fcb-1a95-46e8-a013-c9225ab0397e
[I 2024-08-01 13:04:44,465] Trial 0 finished with value: 6.517470359802246 and parameters: {'x1': 1.987984832464874, 'x2': 2.707693661790487}. Best is trial 0 with value: 6.517470359802246.
[I 2024-08-01 13:04:44,466] Trial 1 finished with value: 106.86405181884766 and parameters: {'x1': -2.8500149993620076, 'x2': 1.2866664600868993}. Best is trial 0 with value: 6.517470359802246.
[I 2024-08-01 13:04:44,467] Trial 2 finished with value: 105.20294189453125 and parameters: {'x1': 2.640273855950176, 'x2': 12.877989234949842}. Best is trial 0 with value: 6.517470359802246.
[I 2024-08-01 13:04:44,469] Trial 3 finished with value: 40.007904052734375 and pa

Best value:  0.3987150192260742
Best params:  {'x1': -3.1289575170956416, 'x2': 12.252442748919297}


In [10]:
import numpy as np
import plotly.graph_objects as go

def test_function(x):
    """
    A simple test function where the optimal solution is outside the range [0, 1].
    """
    if 0 <= x <= 1:
        return 10 + 10 * (x - 0.5)**2  # High penalty within [0, 1]
    else:
        return (x - 2)**2  # Quadratic function with a minimum at x = 2

# Generate x values
x_values = np.linspace(-1, 3, 500)
y_values = [test_function(x) for x in x_values]

# Create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Test Function'))

# Add title and labels
fig.update_layout(
    title='Test Function Plot',
    xaxis_title='x',
    yaxis_title='f(x)',
    showlegend=True
)

# Show the plot
fig.show()


In [21]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from gpytorch.mlls import ExactMarginalLogLikelihood

# Define the test function
def test_function(x):
    if 0 <= x <= 1:
        return 10 + 10 * (x - 0.5)**2  # High penalty within [0, 1]
    else:
        return (x - 2)**2  # Quadratic function with a minimum at x = 2

# Convert test_function to a PyTorch tensor
def objective(x):
    return torch.tensor([test_function(xi.item()) for xi in x], dtype=torch.float32)

# Initialize training data
train_x = torch.tensor([[-0.1], [0.5]], dtype=torch.float32)
train_y = objective(train_x).unsqueeze(-1)

# Run optimization for 10 iterations
for i in range(100):
    # Define and fit the GP model
    gp = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Define the acquisition function
    UCB = UpperConfidenceBound(gp, beta=0.1)

    # Optimize the acquisition function
    bounds = torch.tensor([[-1.0], [3.0]], dtype=torch.float)
    candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )

    # Evaluate the new candidate and add to the training data
    new_y = objective(candidate).unsqueeze(-1)
    train_x = torch.cat([train_x, candidate])
    train_y = torch.cat([train_y, new_y])

    # Print the best candidate found so far
    print(f"Iteration {i+1}")
    print(f"Best candidate: {candidate.item()}")
    print(f"Objective value at best candidate: {test_function(candidate.item())}\n")


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([3.9527]), mean = tensor([7.2050])). Please consider scaling the input to zero mean and unit variance.

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is

Iteration 1
Best candidate: 0.5494778752326965
Objective value at best candidate: 10.024480601375423

Iteration 2
Best candidate: 0.5661972165107727
Objective value at best candidate: 10.043820714737741



/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([2.8064]), mean = tensor([8.6196])). Please consider scaling the input to zero mean and unit variance.

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is

Iteration 3
Best candidate: 0.6710513234138489
Objective value at best candidate: 10.292585552416291

Iteration 4
Best candidate: 0.8650335073471069
Objective value at best candidate: 11.332494614861304

Iteration 5
Best candidate: 1.0640172958374023
Objective value at best candidate: 0.8760636224915288



/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([3.9187]), mean = tensor([8.1399])). Please consider scaling the input to zero mean and unit variance.

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is

...

Iteration 97
Best candidate: 0.9723414778709412
Objective value at best candidate: 12.231064717173048

Iteration 98
Best candidate: 0.9723823666572571
Objective value at best candidate: 12.231451003287113



/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([2.1842]), mean = tensor([11.5561])). Please consider scaling the input to zero mean and unit variance.

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It i

Iteration 99
Best candidate: 0.9725046753883362
Objective value at best candidate: 12.23260668263837



/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_96749/3541269840.py:26: InputDataWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([2.1643]), mean = tensor([11.5695])). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/optim/fit.py:102: OptimizationWarning:

`scipy_minimize` terminated 

Iteration 100
Best candidate: -1.0
Objective value at best candidate: 9.0



In [22]:
train_x[train_y.argmin().item()]

tensor([2.2552])

tensor(8)

In [26]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from gpytorch.mlls import ExactMarginalLogLikelihood
from sklearn.preprocessing import MinMaxScaler

# Define the test function
def test_function(x):
    if 0 <= x <= 1:
        return -(10 + 10 * (x - 0.5)**2)  # High reward within [0, 1]
    else:
        return -(x - 2)**2  # Quadratic function with a maximum at x = 2

# Convert test_function to a PyTorch tensor
def objective(x):
    return torch.tensor([test_function(xi.item()) for xi in x], dtype=torch.float64)

# Initialize training data
train_x = torch.tensor([[-1.0], [0.0], [0.5], [1.0], [3.0]], dtype=torch.float64)
train_y = objective(train_x).unsqueeze(-1)

# Normalize the input data
scaler = MinMaxScaler()
train_x_normalized = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float64)

# Run optimization for 10 iterations
for i in range(10):
    # Define and fit the GP model
    gp = SingleTaskGP(train_x_normalized, train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Define the acquisition function (UCB) which will be maximized to minimize the negative objective
    UCB = UpperConfidenceBound(gp, beta=0.1)

    # Optimize the acquisition function
    bounds = torch.tensor([[0.0], [1.0]], dtype=torch.float64)
    candidate_normalized, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )

    # Denormalize the candidate
    candidate = torch.tensor(scaler.inverse_transform(candidate_normalized), dtype=torch.float64)

    # Evaluate the new candidate and add to the training data
    new_y = objective(candidate).unsqueeze(-1)
    train_x = torch.cat([train_x, candidate])
    train_y = torch.cat([train_y, new_y])

    # Normalize the input data again
    train_x_normalized = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float64)

    # Print the best candidate found so far (remember to invert the sign back)
    print(f"Iteration {i+1}")
    print(f"Best candidate: {candidate.item()}")
    print(f"Objective value at best candidate: {-test_function(candidate.item())}\n")


print()
print()
print()
print()
print()
print()
print()
print()
print()
print()
print()
print("helloooooooo")
print(train_x[train_y.argmin().item()])

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.7302], dtype=torch.float64), mean = tensor([-9.], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.3448], dtype=torch.float64), mean = tensor([-7.6667], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.4913], dtype=torch.float64), mean = tensor([-6.7143], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/pyth

Iteration 1
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 2
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 3
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 4
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 5
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 6
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 7
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 8
Best candidate: 3.0
Objective value at best candidate: 1.0

Iteration 9
Best candidate: 2.9328712050063728
Objective value at best candidate: 0.8702486851300419

Iteration 10
Best candidate: 2.8702769208548724
Objective value at best candidate: 0.7573819189726378












helloooooooo
tensor([0.], dtype=torch.float64)


/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.1385], dtype=torch.float64), mean = tensor([-4.6364], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.0106], dtype=torch.float64), mean = tensor([-4.3333], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.8855], dtype=torch.float64), mean = tensor([-4.0769], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/

In [47]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from botorch.models.transforms.input import Normalize
from gpytorch.mlls import ExactMarginalLogLikelihood

def test_function(x):
    """
    A simple test function where the optimal solution is outside the range [0, 1].
    """
    if 0 <= x <= 1:
        return 10 + 10 * (x - 0.5)**2  # High penalty within [0, 1]
    else:
        return (x - 2)**2  # Quadratic function with a minimum at x = 2

# Convert test_function to a PyTorch tensor
def objective(x):
    return torch.tensor([test_function(xi.item()) for xi in x], dtype=torch.float64)

# Initialize training data
train_x = torch.tensor([[-1.0], [0.0], [0.5], [1.0], [2.0], [3.0]], dtype=torch.float64)

# Normalize the input data
input_transform = Normalize(d=train_x.shape[-1])
train_x_normalized = input_transform(train_x)

train_y = objective(train_x).unsqueeze(-1)


# Run optimization for 10 iterations
for i in range(30):
    # Define and fit the GP model with input normalization
    gp = SingleTaskGP(train_x_normalized, train_y, input_transform=input_transform)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Define the acquisition function (UCB) which will be maximized to minimize the negative objective
    UCB = UpperConfidenceBound(gp, beta=0.1)

    # Optimize the acquisition function
    bounds = torch.tensor([[0.], [1.]], dtype=torch.float64)
    candidate_normalized, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )

    # Denormalize the candidate
    candidate = input_transform.untransform(candidate_normalized)

    # Evaluate the new candidate and add to the training data
    new_y = objective(candidate).unsqueeze(-1)
    train_x = torch.cat([train_x, candidate])
    train_y = torch.cat([train_y, new_y])

    # Normalize the input data again
    train_x_normalized = input_transform(train_x)

    # Print the best candidate found so far (remember to invert the sign back)
    print(f"Iteration {i+1}")
    print(f"Best candidate: {candidate.item()}")
    print(f"Objective value at best candidate: {-test_function(candidate.item())}\n")


print()
print()
print()
print("hellooooo")
print(train_x[train_y.argmin().item()])

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.6036], dtype=torch.float64), mean = tensor([7.5000], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([5.2333], dtype=torch.float64), mean = tensor([7.9176], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not

Iteration 1
Best candidate: 0.29433065100868216
Objective value at best candidate: -10.422998811145124

Iteration 2
Best candidate: -0.3767260425284853
Objective value at best candidate: -5.648826681233115

Iteration 3
Best candidate: 0.37301019040733996
Objective value at best candidate: -10.1612641174038

Iteration 4
Best candidate: 1.2119055557336624
Objective value at best candidate: -0.6210928530834674

Iteration 5
Best candidate: 0.2908516845409481
Objective value at best candidate: -10.43743017859359

Iteration 6
Best candidate: -0.2290501480170043
Objective value at best candidate: -4.968664562374629



/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.8166], dtype=torch.float64), mean = tensor([7.4811], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.6494], dtype=torch.float64), mean = tensor([7.2717], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data 

Iteration 7
Best candidate: 0.3330689222174944
Objective value at best candidate: -10.27865984729629

Iteration 8
Best candidate: 0.42907062914328575
Objective value at best candidate: -10.050309756501292

Iteration 9
Best candidate: 0.3434290201247728
Objective value at best candidate: -10.245144717390888

Iteration 10
Best candidate: 0.5462961407734168
Objective value at best candidate: -10.02143332650512

Iteration 11
Best candidate: 0.3487206941668922
Objective value at best candidate: -10.228854283733469

Iteration 12
Best candidate: 0.6116872268028566
Objective value at best candidate: -10.124740366309128

Iteration 13
Best candidate: 0.35309310432195545
Objective value at best candidate: -10.215816359977598



/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.0880], dtype=torch.float64), mean = tensor([8.1226], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([3.9939], dtype=torch.float64), mean = tensor([8.2339], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data 

...

Iteration 23
Best candidate: 0.48640071751435615
Objective value at best candidate: -10.001849404841243

Iteration 24
Best candidate: 2.782468077000075
Objective value at best candidate: -0.6122562915241951

Iteration 25
Best candidate: 0.4864042806397861
Objective value at best candidate: -10.001848435849217



/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.2761], dtype=torch.float64), mean = tensor([7.8992], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.3994], dtype=torch.float64), mean = tensor([7.6715], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data 

Iteration 26
Best candidate: 2.7824888069712967
Objective value at best candidate: -0.6122887330353634

Iteration 27
Best candidate: 0.48640550167131563
Objective value at best candidate: -10.001848103848086

Iteration 28
Best candidate: 2.782498825965793
Objective value at best candidate: -0.6123044126378444



/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.4538], dtype=torch.float64), mean = tensor([7.5324], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.

/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning:

Data is not standardized (std = tensor([4.4076], dtype=torch.float64), mean = tensor([7.6029], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.



Iteration 29
Best candidate: 0.48640626631510103
Objective value at best candidate: -10.00184789595496

Iteration 30
Best candidate: 2.7825054965672034
Objective value at best candidate: -0.6123148521578855




hellooooo
tensor([2.], dtype=torch.float64)


In [49]:
train_x[train_y.argmin().item()]

tensor([2.], dtype=torch.float64)

In [46]:
train_y

tensor([[ 9.0000],
        [12.5000],
        [10.0000],
        [12.5000],
        [ 1.0000],
        [10.2782],
        [ 0.1178],
        [10.5579],
        [ 7.6001],
        [10.3899],
        [ 0.4752],
        [10.7568],
        [ 0.4935],
        [10.7723],
        [ 0.5052],
        [10.7733],
        [ 0.5060],
        [10.7736],
        [ 0.5062],
        [10.7738],
        [ 0.5063],
        [10.7739],
        [ 0.5063],
        [10.7739],
        [ 0.5063],
        [10.7740],
        [ 0.5064],
        [10.7740],
        [ 0.5064],
        [10.7733],
        [ 0.5064],
        [10.7740],
        [ 0.5064],
        [10.7741],
        [ 0.5025]], dtype=torch.float64)

In [38]:
import numpy as np
import plotly.graph_objects as go

def test_function(x):
    """
    A simple test function where the optimal solution is outside the range [0, 1].
    """
    if 0 <= x <= 1:
        return 10 + 10 * (x - 0.5)**2  # High penalty within [0, 1]
    else:
        return (x - 2)**2  # Quadratic function with a minimum at x = 2

# Generate x values
x_values = np.linspace(-1, 3, 500)
y_values = [test_function(x) for x in x_values]

# Create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Test Function'))

# Add title and labels
fig.update_layout(
    title='Test Function Plot',
    xaxis_title='x',
    yaxis_title='f(x)',
    showlegend=True
)

# Show the plot
fig.show()
